

# Recommendation Engine

#### **Note: Kindly run this notebook on Google Colab to avoid Memory issues.**

<a id=section1></a>
## 1. Matrix Factorization

**Most important technique in recommendation system**<br><br>
- When a user gives feedback to a cerrtain movie they saw, this collection of feedback can be collected in the form of a matrix.
- Each row represents each users,
- Each column represents different movies.
- The matrix will be sparse since not everyone is going to watch every movies.

<img src = "https://raw.githubusercontent.com/insaid2018/Term-4/master/images/rec14.png">

The idea behind such models is that the preference of a user can be determined by a small number of hidden factors. We can call these factors as **Embeddings**.<br><br>

<a id=section2></a>
## 2. Let's understand what is SVD

Singular Value Decomposition(SVD) is a variability localization technique in which we represent data in form of matrix and then reduce the number of columns it has in order to maximize loss of dimensionality while minimizing loss of variability in the data being processed.<br>
Why wouldn’t the data be lost? The answer for that question is the essence of SVD.

Basically, SVD breaks a matrix into three other matrices called u, v, and d.

1- A is the real matrix with m*n elements.

2- U is an Orthogonal matrix with m*m elements

3- V is an Orthogonal matrix with n*n elements.

4- D is a diagonal matrix with m*n elements.

Orthogonal matrix is a matrix that does not get its properties changed if multiplied by other numbers.



<a id=section3></a>
## 3. What is the use of it?

When we decompose our matrix A into (U, D, V), a few left-most columns of all three matrices represent almost all the information we need to recover our actual data. For example 92% of the information in just 5% of total columns which is a pretty good deal given that you have reduced the size of your data set tremendously.

This means that SVD found some relation between all the columns of the matrix A and represented this same information with fewer columns.

The curse of dimensionality is no longer able to affect your performance.

**Matrix decompostion can be formulated as  an optimization problem with loss functions and constraints**

We can understand embeddings as low dimensional hidden factors for items and users.<br>
Let's say, we have 5 dimensional (D or n_factors = 5) embeddings for both items and users. Then for user-X and movie-A, we can say those 5 numbers might represent 5 different characterestics about the movies, like:
- How much movie-A is sci-fi intense?
- How recent is the movie?
- How much special effects ar in movie?
- How dialogue drive is the movie?


Like wise some numbers in user embedding matrix might represents,
- How much does user-X like sci-fi movies?
- How much does user-X like recent movies?



<a id=section4></a>
## 4. Loading Data

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3156221 sha256=d4e10121ddc17ddc74933d751e8a9fc1ee1fd56eed417a093057adc582fa258a
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reading ratings file
# Ignore the timestamp column
ratings = pd.read_csv('https://raw.githubusercontent.com/insaid2018/Term-4/master/Data/Assignment/ratings.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'movie_id', 'rating'])
#ratings = pd.read_csv('ratings.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'movie_id', 'rating'])


# Reading movies file
#movies = pd.read_csv('movies.csv', sep='\t', encoding='latin-1', usecols=['movie_id', 'title', 'genres'])
movies = pd.read_csv('https://raw.githubusercontent.com/insaid2018/Term-4/master/Data/Assignment/movies.csv', sep='\t', encoding='latin-1', usecols=['movie_id', 'title', 'genres'])

<a id=section5></a>
## 5. Implementing Singular Vector Decomposition

#### Using Ratings Data

In [3]:
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print('Number of users ='+ str(n_users) +' | Number of Movies =' +str(n_movies))

Number of users =6040 | Number of Movies =3706


- We want the format of my ratings matrix to be one row per user and one column per movie.
- We'll pivot *ratings* to get that and call the new variable *Ratings* (with a capital *R).

In [4]:
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [5]:
#Creating the Pivot
Ratings= ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
Ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We need to de-normalize the data (normalize by each users mean) and convert it from a dataframe to a numpy array.

In [6]:
R = Ratings.values
user_ratings_mean = np.mean(R, axis=1)
print(user_ratings_mean)
print('----------------')
print(user_ratings_mean.reshape(-1,1))
print('----------------')
print(R)
print('----------------')
Ratings_demanded= R - user_ratings_mean.reshape(-1,1)
print(Ratings_demanded)

[0.05990286 0.12924987 0.05369671 ... 0.02050729 0.1287102  0.3291959 ]
----------------
[[0.05990286]
 [0.12924987]
 [0.05369671]
 ...
 [0.02050729]
 [0.1287102 ]
 [0.3291959 ]]
----------------
[[5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [3. 0. 0. ... 0. 0. 0.]]
----------------
[[ 4.94009714 -0.05990286 -0.05990286 ... -0.05990286 -0.05990286
  -0.05990286]
 [-0.12924987 -0.12924987 -0.12924987 ... -0.12924987 -0.12924987
  -0.12924987]
 [-0.05369671 -0.05369671 -0.05369671 ... -0.05369671 -0.05369671
  -0.05369671]
 ...
 [-0.02050729 -0.02050729 -0.02050729 ... -0.02050729 -0.02050729
  -0.02050729]
 [-0.1287102  -0.1287102  -0.1287102  ... -0.1287102  -0.1287102
  -0.1287102 ]
 [ 2.6708041  -0.3291959  -0.3291959  ... -0.3291959  -0.3291959
  -0.3291959 ]]


- With the ratings matrix properly formatted and normalized, we can do some dimensionality reduction.

<a id=section6></a>
## 6. Setting Up SVD

Scipy and Numpy both have functions to do the singular value decomposition. We're going to use the Scipy function *svds* because it let's us choose how many latent factors we want to use to approximate the original ratings matrix (instead of having to truncate it after).

In [7]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(Ratings_demanded, k=50)

As we're going to leverage matrix multiplication to get predictions, We'll convert the $\Sigma$ (now are values) to the diagonal matrix form.

In [8]:
sigma = np.diag(sigma)

In [9]:
print(U)
print(U.shape)
print('-----------------------')
print(sigma)
print('-----------------------')
print(sigma.shape)
print('-----------------------')
print(Vt)
print('----------------')
print(Vt.shape)



[[-4.97801875e-03  5.86971868e-03 -1.18186843e-02 ... -2.95139320e-03
  -1.95703358e-03  5.46889776e-03]
 [-1.10526375e-03 -4.04545890e-03  1.16776791e-02 ... -9.18855171e-04
   2.17034433e-03  1.04359614e-02]
 [ 9.44963839e-03 -1.43519545e-02 -3.93638119e-04 ...  2.89764529e-03
   2.86504507e-03  6.13985002e-03]
 ...
 [-1.05731072e-02 -6.80807641e-03 -2.63392883e-03 ...  4.84286245e-05
  -1.89077440e-03  1.52456048e-03]
 [ 6.34420788e-03 -9.45269844e-03  2.69929558e-03 ...  1.07208981e-02
  -1.88878158e-02  6.87143535e-03]
 [-1.84854679e-02  1.28388950e-02  8.46988257e-03 ...  1.89987575e-03
  -4.15563933e-02  1.92850979e-02]]
(6040, 50)
-----------------------
[[ 147.18581225    0.            0.         ...    0.
     0.            0.        ]
 [   0.          147.62154312    0.         ...    0.
     0.            0.        ]
 [   0.            0.          148.58855276 ...    0.
     0.            0.        ]
 ...
 [   0.            0.            0.         ...  574.46932602
     0.

<a id=section7></a>
## 7. Making Recommendations using SVD

Now, we have everything we need to make movie ratings predictions for every user. We can do it all at once by following the math and matrix multiply $U$, $\Sigma$, and $V^{T}$ back to get the rank $k=50$ approximation of $A$.

But first, we need to add the user means back to get the actual star ratings prediction.

In [10]:
# Performing the Dot Product of U, sigma, and Vt , w e get the ratings in a decreasing order of importance
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)+ user_ratings_mean.reshape(-1,1)

In [11]:
all_user_predicted_ratings

array([[ 4.28886061,  0.14305516, -0.1950795 , ...,  0.03191195,
         0.05044975,  0.08891033],
       [ 0.74471587,  0.16965927,  0.33541808, ..., -0.10110207,
        -0.0540982 , -0.14018846],
       [ 1.81882382,  0.45613623,  0.09097801, ...,  0.01234452,
         0.01514752, -0.10995596],
       ...,
       [ 0.61908871, -0.16176859,  0.10673806, ..., -0.01336948,
        -0.0303543 , -0.11493552],
       [ 1.50360483, -0.03620761, -0.16126817, ..., -0.01090407,
        -0.03864749, -0.16835943],
       [ 1.99624816, -0.18598715, -0.1564782 , ..., -0.00664061,
         0.12706713,  0.28500112]])

With the predictions matrix for every user, we can build a function to recommend movies for any user. We return the list of movies the user has already rated, for the sake of comparison.

In [12]:
pred= pd.DataFrame(all_user_predicted_ratings, columns= Ratings.columns)
pred.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.288861,0.143055,-0.195080,-0.018843,0.012232,-0.176604,-0.074120,0.141358,-0.059553,-0.195950,...,0.027807,0.001640,0.026395,-0.022024,-0.085415,0.403529,0.105579,0.031912,0.050450,0.088910
1,0.744716,0.169659,0.335418,0.000758,0.022475,1.353050,0.051426,0.071258,0.161601,1.567246,...,-0.056502,-0.013733,-0.010580,0.062576,-0.016248,0.155790,-0.418737,-0.101102,-0.054098,-0.140188
2,1.818824,0.456136,0.090978,-0.043037,-0.025694,-0.158617,-0.131778,0.098977,0.030551,0.735470,...,0.040481,-0.005301,0.012832,0.029349,0.020866,0.121532,0.076205,0.012345,0.015148,-0.109956
3,0.408057,-0.072960,0.039642,0.089363,0.041950,0.237753,-0.049426,0.009467,0.045469,-0.111370,...,0.008571,-0.005425,-0.008500,-0.003417,-0.083982,0.094512,0.057557,-0.026050,0.014841,-0.034224
4,1.574272,0.021239,-0.051300,0.246884,-0.032406,1.552281,-0.199630,-0.014920,-0.060498,0.450512,...,0.110151,0.046010,0.006934,-0.015940,-0.050080,-0.052539,0.507189,0.033830,0.125706,0.199244


 Just to recall, below are the samples of movies and ratings dataset

In [13]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
def recommended_movies( predictions, user_ID, movies, original_ratings, num_recommendations):
    #Get and sort suer's predictions
    user_row_number = user_ID-1 # User_id starts at 1 and not 0;
    sorted_user_predictions = predictions.iloc[user_row_number].sort_values(ascending=False)

    #Get the user's data and merge in the movie information
    user_data =  original_ratings[original_ratings.user_id==(user_ID)]
    user_full= (user_data.merge(movies, how='left', left_on='movie_id', right_on='movie_id').sort_values(['rating'], ascending=False))

    print('User {0} has already rated {1} movies.'.format(user_ID, user_full.shape[0]))
    print('Recommending highest {0} predicted ratings not already rated.'.format(num_recommendations))

    # Recommend the highest predicted rating movies that the user hasn't not seen yet.

    recommendations = (movies[~ movies['movie_id'].isin(user_full['movie_id'])].
                      merge(pd.DataFrame(sorted_user_predictions).reset_index(), how='left',
                            left_on='movie_id',
                            right_on= 'movie_id').
                       rename(columns={user_row_number: 'Predictions'}).
                       sort_values('Predictions', ascending=False).
                       iloc[:num_recommendations, :-1]



                      ) #movies not seen by user yet
    return user_full, recommendations



In [15]:
#Line by Line approach of the above algorithm
#user_row_number = user_ID-1 # User_id starts at 1 and not 0; user_id = 1310
user_row_number = 1310-1


In [16]:
sorted_user_predictions = pred.iloc[user_row_number].sort_values(ascending=False)
sorted_user_predictions

movie_id
1097    1.571580
1090    1.373596
1674    1.292395
1196    1.263977
1961    1.153507
          ...   
2712   -0.334191
3578   -0.362995
910    -0.368467
1304   -0.494537
1136   -0.512347
Name: 1309, Length: 3706, dtype: float64

In [17]:
user_data =  ratings[ratings.user_id==(1310)]
user_data.head()


,user_id,movie_id,rating
215928,1310,2988,3
215929,1310,1293,5
215930,1310,1295,2
215931,1310,1299,4
215932,1310,2243,4


In [18]:
user_full= (user_data.merge(movies, how='left', left_on='movie_id', right_on='movie_id').sort_values(['rating'], ascending=False))
user_full.head()

,user_id,movie_id,rating,title,genres
5,1310,2248,5,Say Anything... (1989),Comedy|Drama|Romance
6,1310,2620,5,This Is My Father (1998),Drama|Romance
7,1310,3683,5,Blood Simple (1984),Drama|Film-Noir
15,1310,1704,5,Good Will Hunting (1997),Drama
1,1310,1293,5,Gandhi (1982),Drama


In [19]:
(movies[~ movies['movie_id'].isin(user_full['movie_id'])].
                      merge(pd.DataFrame(sorted_user_predictions).reset_index(), how='left',
                            left_on='movie_id',
                            right_on= 'movie_id').
                       rename(columns={user_row_number: 'Predictions'}).
                       sort_values('Predictions', ascending=False).
                       iloc[:20, :-1])


,movie_id,title,genres
1618,1674,Witness (1985),Drama|Romance|Thriller
1880,1961,Rain Man (1988),Drama
1187,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1216,1242,Glory (1989),Action|Drama|War
1202,1225,Amadeus (1984),Drama
1273,1302,Field of Dreams (1989),Drama
1220,1246,Dead Poets Society (1989),Drama
1881,1962,Driving Miss Daisy (1989),Drama
1877,1957,Chariots of Fire (1981),Drama
1938,2020,Dangerous Liaisons (1988),Drama|Romance


Now, we write a function to return the movies with the highest predicted rating that the specified user hasn't already rated. Though we didn't use any explicit movie content features (such as genre or title), we'll merge in that information to get a more complete picture of the recommendations.

In [20]:
movies[~ movies['movie_id'].isin(user_full['movie_id'])].merge(pd.DataFrame(sorted_user_predictions).reset_index(), how='left',
                            left_on='movie_id',
                            right_on= 'movie_id').rename(columns={user_row_number: 'Predictions'}).sort_values('Predictions', ascending=False).iloc[:20,:-1]


,movie_id,title,genres
1618,1674,Witness (1985),Drama|Romance|Thriller
1880,1961,Rain Man (1988),Drama
1187,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1216,1242,Glory (1989),Action|Drama|War
1202,1225,Amadeus (1984),Drama
1273,1302,Field of Dreams (1989),Drama
1220,1246,Dead Poets Society (1989),Drama
1881,1962,Driving Miss Daisy (1989),Drama
1877,1957,Chariots of Fire (1981),Drama
1938,2020,Dangerous Liaisons (1988),Drama|Romance


Let's try to recommend 20 movies for user with ID 1310.

In [21]:
already_rated, recommendations = recommended_movies(pred, 1310, movies,ratings, 20 )

User 1310 has already rated 24 movies.
Recommending highest 20 predicted ratings not already rated.


In [22]:
already_rated

,user_id,movie_id,rating,title,genres
5,1310,2248,5,Say Anything... (1989),Comedy|Drama|Romance
6,1310,2620,5,This Is My Father (1998),Drama|Romance
7,1310,3683,5,Blood Simple (1984),Drama|Film-Noir
15,1310,1704,5,Good Will Hunting (1997),Drama
1,1310,1293,5,Gandhi (1982),Drama
12,1310,3101,4,Fatal Attraction (1987),Thriller
11,1310,1343,4,Cape Fear (1991),Thriller
20,1310,2000,4,Lethal Weapon (1987),Action|Comedy|Crime|Drama
18,1310,3526,4,Parenthood (1989),Comedy|Drama
17,1310,3360,4,Hoosiers (1986),Drama


In [23]:
recommendations

,movie_id,title,genres
1618,1674,Witness (1985),Drama|Romance|Thriller
1880,1961,Rain Man (1988),Drama
1187,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1216,1242,Glory (1989),Action|Drama|War
1202,1225,Amadeus (1984),Drama
1273,1302,Field of Dreams (1989),Drama
1220,1246,Dead Poets Society (1989),Drama
1881,1962,Driving Miss Daisy (1989),Drama
1877,1957,Chariots of Fire (1981),Drama
1938,2020,Dangerous Liaisons (1988),Drama|Romance


- It's good to see that, although we didn't actually use the genre of the movie as a feature, the truncated matrix factorization features "picked up" on the underlying tastes and preferences of the user.


- We've recommended some comedy, drama, and romance movies - all of which were genres of some of this user's top rated movies.

<a id=section8></a>
## 8. Model Evaluation

We will use the [Surprise](https://pypi.python.org/pypi/scikit-surprise) library that provided various ready-to-use powerful prediction algorithms including (SVD) to evaluate its **RMSE (Root Mean Squared Error)** on the MovieLens dataset. It is a Python scikit building and analyzing recommender systems.

In [24]:
#Import libraries from surprise Package
!pip install surprise
from surprise import Reader, Dataset, SVD

In [25]:
#Load Reader Library
reader = Reader()
#Load ratings dataset with Dataset library
data = Dataset.load_from_df(ratings[['user_id','movie_id','rating']], reader)
data

In [26]:
from surprise.model_selection import cross_validate

# Use the SVD algorithm
svd=SVD()

 # Compount the RMSE of the SVD algorithm
cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=False)


{'test_rmse': array([0.87553242, 0.87446241, 0.87562437, 0.87168014, 0.8735074 ]),
 'fit_time': (14.835643768310547,
  10.56289291381836,
  10.617872714996338,
  10.534020900726318,
  10.882707595825195),
 'test_time': (1.730215072631836,
  1.5578207969665527,
  1.5010170936584473,
  1.5425574779510498,
  1.484935998916626)}

- Root Mean Square Error of 0.8736 which is pretty good.


- Now train on the dataset and arrive at predictions.

In [27]:
trainset = data.build_full_trainset()
svd.fit(trainset)

We'll pick again user with ID 1310 and check the ratings he has given.

In [28]:
ratings[ratings['user_id']==1310][:5]

,user_id,movie_id,rating
215928,1310,2988,3
215929,1310,1293,5
215930,1310,1295,2
215931,1310,1299,4
215932,1310,2243,4


Now let's use SVD to predict the rating that User with ID 1310 will give to a random movie (let's say with Movie ID 1994).

In [29]:
svd.predict(1310, 1293)

Prediction(uid=1310, iid=1293, r_ui=None, est=3.8218154027137032, details={'was_impossible': False})

For movie with ID 1994, we get an estimated prediction of 3.349. The recommender system works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie.

<a id=section9></a>
# 9. Conclusion

In this notebook, we attempted to build a movie recommendation sytem based on latent features from a low rank matrix factorization method called SVD. As it captures the underlying features driving the raw data, it can scale significantly better to massive datasets as well as make better recommendations based on user's tastes.

However, we still likely lose some meaningful signals by using a low-rank approximation. Specifically, there's an interpretability problem as a singular vector specifies a linear combination of all input columns or rows. There's also a lack of sparsity when the singular vectors are quite dense. Thus, SVD approach is limited to linear projections.